<a href="https://colab.research.google.com/github/vivek09thakur/Fine-Tuning-GPT2/blob/main/Colab%20Notebook/GPT2_FineTuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fine Tuning GPT-2**

In [1]:
%pip install --upgrade torch
%pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.5 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")

## **Fine Tuning**

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import AdamW, get_linear_schedule_with_warmup
import torch

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

sentence = "Hello, my name is Saturday"

tokens = tokenizer.encode(sentence, return_tensors='pt')

inputs = tokens[:,:-1]
labels = tokens[:,1:]

epochs = 5
learning_rate = 1e-5

device = torch.device("cuda" if torch.cuda.is_available() else "mps")
model.to(device)

optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(inputs) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
model.train()

for epoch in range(epochs):
    for i in range(len(inputs)):
        optimizer.zero_grad()
        outputs = model(inputs[i].to(device), labels=labels[i].to(device))
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f"Epoch {epoch + 1} completed")

model.save_pretrained("my_fine_tuned_model")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Epoch 1 completed
Epoch 2 completed
Epoch 3 completed
Epoch 4 completed
Epoch 5 completed


## **TESTING THE FINE TUNED MODEL**

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("my_fine_tuned_model")

def generate_response(prompt_text):
    encoded_prompt = tokenizer.encode(
        prompt_text, add_special_tokens=False, return_tensors="pt"
    )

    tokenizer.pad_token = tokenizer.eos_token
    attention_mask = encoded_prompt.ne(tokenizer.pad_token_id).long()

    output_sequences = model.generate(
        input_ids=encoded_prompt,
        attention_mask=attention_mask,
        max_length=100,
        temperature=1.0,
        top_k=0,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,  # use EOS token for padding
    )

    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequence = output_sequences[0].tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    return text

In [5]:
while True:
    try:
        user_input = input("> ")
        response = generate_response(prompt_text=user_input)
        print(">_ ",response)
    except KeyboardInterrupt:
        break

> hello
>_  hello action name: SealTime 1 Action a vm VERSION dest VERSION

This allows all sciences of the type:University,Coachella,DeviantArt,US Digital Domain Admin,Membership SCOPE statements are considered true if they are true if that their values are known DEFAULT switch STATUS reCheck Note 1 In this case, the node, which runs, will not accept information outside of article statements while using this node UPDATE STATUS reCheck Note ~ ecst smart
> what did you said?
>_  what did you said? Daughter, Son, sister? Did you torture her? He put you under my bed as he stroked my body. Then frightening how he made me feel cracklish cow mouth sensitive and ungracious tongue gripping my sticky abramed neck you know? Let silky hand of him thrusts all I need for you to squeeze my belly hard and tight and purr finish the taste of your whit hot hard boiled tongue< 2 Then tried to resist by brained hand as
> you are gone mad
>_  you are gone mad, you're a lump of shit."

Henro Tudela Real Est